<a href="https://colab.research.google.com/github/dhairyabhuta1008/MM_probit/blob/main/mm_with_gr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import *
import numpy as np
import pandas as pd
# from utils import check_for_empty_cols_in_summary, get_zone_output_path
import os
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import shortest_path
import sys
from itertools import combinations
import sympy
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
zone = '500.0'
cap_mode = 1                    # 2 for all
# cut = -1                      # -1 for no choice
# arrival = -1                  # -1 for no choice
root = 'data'
results_path = 'results/regression'


In [ ]:
def get_zone_output_path(zone, folder):
    """
    Returns the output folder name and output file prefix for a zone
    Args:
        zone (str): Zone number in float string
        folder (str): Root data folder
    Returns:
        output folder name and output file prefix
    """
    return os.path.join(folder, 'Zones', zone[:-2]), 'Zone_'+zone[:-2] + '_'

In [ ]:
def MMfeatures(Location,filename,summary, slotsOffered):
    betacoef=[append([0],np.random.rand(len(slotsOffered)+2))]
    [featuresleveldf, discountslots, ecoslots]=getActiveFeatures(summary,slotsOffered)
    # betadfnew=pd.DataFrame([np.random.rand(len(slotsOffered)+3)], columns=[col+'_Asc' for col in ['NO_PURCHASE']+slotsOffered]+['Discount','Eco'])
    betanew=betacoef[0][:]
    # colbetaext= [col+'_Asc' for col in ['NO_PURCHASE']+slotsOffered] + discountslots+ecoslots
    # betaextdfnew=pd.DataFrame(columns=colbetaext)
    betaextnew = formBetaExtNumpy(betanew, len(discountslots), len(ecoslots))
    designdf = getDesignMatrix(featuresleveldf.columns.tolist(), slotsOffered)
    assortmentdf = summary.loc[:,['C_'+col for col in ['NO_PURCHASE']+slotsOffered]].fillna(0)
    choicedf = summary.loc[:,[col for col in ['NO_PURCHASE']+slotsOffered]].fillna(0)
    design = designdf.values
    featureslevel = featuresleveldf.values
    assortment = assortmentdf.values
    choice = choicedf.values
    i = 0
    while True:
        i += 1
        # pdb.set_trace()
        beta=np.copy(betanew)
        betaext=np.copy(betaextnew)
        [betanew, Q] = updateBetaNumpy(design, featureslevel, discountslots, ecoslots, assortment, choice, betaext, beta, slotsOffered)
        loglikeli = sum(log(sum(Q*choice,1)))
        betaextnew = formBetaExtNumpy(betanew, len(discountslots), len(ecoslots))
        print ('Iteration=',i, 'loglikelihood =',loglikeli, 'beta_disc', betanew[-2], 'beta_eco', betanew[-1])
        if np.linalg.norm(betanew-beta)< 10**-6 or i > 500:
            predictprobdf = pd.DataFrame(Q,columns=['NO_PURCHASE']+slotsOffered)
            betadf = pd.DataFrame([np.array(betanew)], columns=['NO_PURCHASE']+slotsOffered+['Discount', 'Eco'])
            predictprobdf.to_csv(Location+filename+'predprobfeatures.csv')
            betadf.to_csv(Location+filename+'betafeatures.csv')
            del summary, predictprobdf, designdf, featuresleveldf, assortmentdf, choicedf, design, featureslevel, assortment, choice
            break
    return betadf.iloc[0]

In [ ]:
def updateBetaNumpy(design, featureslevel, discountslots, ecoslots, grslots, assortment, choice, betaext, beta, slotsOffered):
    #pdb.set_trace()
    betanew = np.zeros(len(slotsOffered)+4)
    featuresutil = design*betaext ### k*l
    utils = featureslevel.dot(featuresutil.T)
    exputils = np.exp(utils)*(assortment)
    Q = exputils/exputils.sum(axis=1, keepdims=True)

    predsharesext = sum(Q.dot(design)*featureslevel, 0)
    truesharesext = sum(choice.dot(design)*featureslevel, 0)
    #check
    trueshare_disc = sum(truesharesext[len(slotsOffered)+1:len(slotsOffered)+1+len(discountslots)])
    trueshare_eco = sum(truesharesext[len(slotsOffered)+1+len(discountslots):len(slotsOffered)+1+len(discountslots)+len(ecoslots)])
    trueshare_gr = sum(truesharesext[len(slotsOffered)+1+len(discountslots)+len(ecoslots):len(slotsOffered)+1+len(discountslots)+len(ecoslots)+len(grslots)])
    #print("Trueshare GR")
    #print(trueshare_gr)

    predshare_disc = sum(predsharesext[len(slotsOffered)+1:len(slotsOffered)+1+len(discountslots)])
    predshare_eco = sum(predsharesext[len(slotsOffered)+1+len(discountslots):len(slotsOffered)+1+len(discountslots)+len(ecoslots)])
    predshare_gr = sum(predsharesext[len(slotsOffered)+1+len(discountslots)+len(ecoslots):len(slotsOffered)+1+len(discountslots)+len(ecoslots)+len(grslots)])
    #print("Predshare GR")
    #print(predshare_gr)

    trueshare = np.append(truesharesext[:len(slotsOffered)+1], [trueshare_disc, trueshare_eco, trueshare_gr])
    predictshare = np.append(predsharesext[:len(slotsOffered)+1], [predshare_disc, predshare_eco, predshare_gr])
    #print("Trueshare")
    #print(trueshare)
    #print("predshare")
    #print(predictshare)
    #truseahre
    betanew[1:] = beta[1:][:] + np.log(trueshare[1:][:]) - np.log(predictshare[1:][:])
    if not discountslots:
        betanew[-3] = 0
    if not ecoslots:
        betanew[-2] = 0
    if not grslots:
        betanew[-1] = 0
    #beta=np.append(beta[:len(slotsOffered)+1]-beta[0],beta[-2:])
    return [betanew, Q]

In [ ]:
def getLocation(zone, computername):
    Location= computername+r'\\Zones'+'\\'+zone[:-2]+ '\\'
    Filename= 'Zone_'+zone[:-2]+'_'
    return [Location,Filename]

#Global Min(non-negative) and max of all the Gr slots and use that as the range.
#make global_normalize() after we check this new approach
# def normalize(summary, grslots):
#     print("normalizing")
#     for grslot in grslots:
#         if summary[grslot].min() < 0:
#             summary[grslot] = (summary[grslot] - summary[grslot].min()) / (summary[grslot].max() - summary[grslot].min())
#         else:
#             summary[grslot] = summary[grslot] / summary[grslot].max()
#         #print(grslot + " min: "+ str(summary[grslot].min()) + " sum: "+ str(summary[grslot].sum()))

def normalize(summary, grslots):
    print("normalizing")
    max_array = []
    min_array = []
    for grslot in grslots:
        max_array.append(summary[grslot].max())
        min_array.append(summary[grslot].min())
    max_value = max(max_array)
    min_value = min(min_array)
    print(max_value)
    print(min_value)
    for grslot in grslots:
        # print(summary[grslot].min())
        # print(summary[grslot].max())
        # if(summary[grslot].max() == summary[grslot].min()):
        #     summary[grslot] = 0
        # else:
        #     summary[grslot] = (summary[grslot] - summary[grslot].min())
        if(max_value == min_value):
            summary[grslot] = summary[grslot] / max_value
        else:
            summary[grslot] = (summary[grslot] - min_value) / (max_value - min_value)
        #  for index, row in summary.iterrows():
        #     if(row[grslot]<0):
        #          summary[grslot][index] = 0
        #     else:
        #          summary[grslot][index] = 1
        # print(summary[grslot])
        # print(summary[grslot].min())
        # print(summary[grslot].max())
        #print(grslot + " min: "+ str(summary[grslot].min()) + " sum: "+ str(summary[grslot].sum()))

def formBetaExt(betadfnew,betaextdfnew):
    betaextdfnew[[col for col in betaextdfnew.columns if col.endswith('Asc')]] = betadfnew[[col for col in betadfnew.columns if col.endswith('Asc')]]
    betaextdfnew[[col for col in betaextdfnew.columns if col.endswith('Discount')]] = betadfnew['Discount'][0]
    betaextdfnew[[col for col in betaextdfnew.columns if col.endswith('Eco')]] = betadfnew['Eco'][0]
    return  betaextdfnew


def formBetaExtNumpy(beta,len_disc,len_eco, len_gr):
    betaext_Asc= beta[0:-3][:]
    beta_disc=beta[-3]
    beta_eco=beta[-2]
    beta_gr=beta[-1]
    betaext_disc= beta_disc*np.ones(len_disc)
    betaext_eco=beta_eco*np.ones(len_eco)
    betaext_gr=beta_gr*np.ones(len_gr)
    betaextnew=np.append(betaext_Asc,betaext_disc)
    betaextnew=np.append(betaextnew,betaext_eco)
    betaextnew=np.append(betaextnew,betaext_gr)
    return betaextnew

In [ ]:
def getActiveFeatures(summary,slotsOffered):
    featuresleveldf = summary.loc[:,[col+'_Discount' for col in slotsOffered] + [col+'_Eco' for col in slotsOffered] + [col+'Gr' for col in slotsOffered]]
    # this is because gr has +ve and -ve that's why we use != 0
    featuresleveldf = featuresleveldf.loc[:, sum(featuresleveldf, 0) != 0]
    ecoslots= [col for col in featuresleveldf.columns if col.endswith('Eco')]
    discountslots= [col for col in featuresleveldf.columns if col.endswith('Discount')]
    grslots= [col for col in featuresleveldf.columns if col.endswith('Gr')]
    featuresleveldf=summary.loc[:,discountslots+ecoslots+grslots]
    featuresleveldf.loc[:, grslots] += abs(featuresleveldf.loc[:, grslots].min())             # gr value non negative (check)
    for i in reversed(['NO_PURCHASE']+slotsOffered):
        featuresleveldf.insert(0, i+'_Asc', value=1)
    return [featuresleveldf, discountslots, ecoslots, grslots]


In [ ]:
def getDesignMatrix(featurescolumns, slotsOffered):
    '''without no purchase
    df=pd.DataFrame(columns=featurescolumns, index=slotsOffered)
    for i in slotsOffered:
        df.loc[i, [col for col in featurescolumns if i in col]]=1
    return df.fillna(0)'''

    #with no purchase
    df=pd.DataFrame(columns=featurescolumns, index=np.append(['NO_PURCHASE'], slotsOffered))
    for i in np.append(['NO_PURCHASE'],slotsOffered):
        df.loc[i, [col for col in featurescolumns if i in col]]=1

    return df.fillna(0)

In [ ]:
def MMfeaturesBoot(Location, filename, summary, slotsOffered):

    # 3 -> beta for disc, eco, Gr , np.append(0) => for NO_PURCHASE

    betacoef = [np.append([0], np.random.rand(len(slotsOffered) + 3))]
    [featuresleveldf, discountslots, ecoslots, grslots] = getActiveFeatures(summary, slotsOffered)
    featuresleveldf.to_csv('featuresleveldf.csv')
    normalize(summary, grslots)
    betanew = betacoef[0][:]
    betaextnew = formBetaExtNumpy(betanew, len(discountslots), len(ecoslots), len(grslots))
    designdf = getDesignMatrix(featuresleveldf.columns.tolist(), slotsOffered)

    #without no purchase
    #which time slot customer had available
    #assortmentdf = summary.loc[:, ['C_' + col for col in  slotsFiltered]].fillna(0)
    #which time slot customer chooses
    #choicedf = summary.loc[:, [col for col in slotsFiltered]].fillna(0)

    #with no purchase
    assortmentdf = summary.loc[:, ['C' + col for col in ['NO_PURCHASE'] + slotsOffered]].fillna(0)
    choicedf = summary.loc[:, [col for col in ['NO_PURCHASE'] + slotsOffered]].fillna(0)

    design = designdf.values
    featureslevel = featuresleveldf.values
    # featureslevel = featureslevel/(3*featureslevel.max())
    assortment = assortmentdf.values
    choice = choicedf.values

    C = np.where(choice == 1)[1]
    membership = assortment
    nprods = assortment.shape[1]
    ## check if the MM algorithm would coverge by testing if the item-item graph
    # is strongly connected
    row = []
    col = []
    data = []
    for i in range(membership.shape[0]):
        assort = list(np.nonzero(membership[i, :])[0])
        try:
            assort.remove(C[i])
        except ValueError:
            print (i, C[i], assort)
            break
        row += len(assort)*[C[i]]
        col += assort
        data += len(assort)*[1]

    dist_matrix = csr_matrix( (data, (row, col)), shape=(nprods, nprods) )
    #changed the scipy package
    Z = shortest_path(csgraph = dist_matrix, method='D') # Dijkstra's algorithm
    I = np.eye(nprods)
    if np.count_nonzero(I+Z) < nprods**2:
        # condition for convergence of MM algo not met
        sys.stderr.write('Warning: Convergence condition for MM algorithm not met...adding noise to the data matrix...\n')
        pairs = [pair for pair in combinations(np.delete(np.arange(nprods),0), 2)]
        npairs = len(pairs)
        pairs = np.array(pairs)
        pairs = np.tile(pairs, (2, 1))
        Z = np.zeros((len(pairs), nprods))
        for i,pair in enumerate(pairs): Z[i, pair] = 1
        assortment = np.vstack((assortment, Z))
        d = np.append(pairs[:npairs, 0],pairs[npairs:, 1])
        choicenew=np.zeros((Z.shape[0],nprods))
        choicenew[np.arange(Z.shape[0]),d] = 1
        choice = np.vstack((choice, choicenew))
        featuresnew=np.zeros((Z.shape[0], featureslevel.shape[1]))
        featuresnew[:, np.arange(nprods)] = 1
        featureslevel = np.vstack((featureslevel, featuresnew))
    i = 0
    while True:
        i += 1
        # pdb.set_trace()
        beta = np.copy(betanew)
        betaext = np.copy(betaextnew)
        [betanew, Q] = updateBetaNumpy(design, featureslevel, discountslots, ecoslots, grslots, assortment, choice, betaext,
                                       beta, slotsOffered)
        sum_for_log = sum(Q * choice, 1)
        log_of_sum = np.where(sum_for_log != 0, np.log(sum_for_log), 0)
        #print("sum_for_log: {}".format(sum_for_log))
        #print("log of sum: {}".format(log(sum_for_log)))

        loglikeli = sum(log_of_sum)
        betaextnew = formBetaExtNumpy(betanew, len(discountslots), len(ecoslots), len(grslots))
        print('Iteration=', i, 'loglikelihood =', loglikeli, 'beta_disc', betanew[-3], 'beta_eco', betanew[-2],'beta_gr', betanew[-1] )
        print(np.linalg.norm(betanew - beta))
        if np.linalg.norm(betanew - beta) < 10 ** -3 or i > 1000:
            #without no purchase
            #predictprobdf = pd.DataFrame(Q, columns=slotsOffered)
            #betadf = pd.DataFrame([np.array(betanew)], columns=slotsOffered + ['Discount', 'Eco', 'Gr'])

            #with no purchase
            predictprobdf = pd.DataFrame(Q, columns=['NO_PURCHASE'] + slotsOffered)
            betadf = pd.DataFrame([np.array(betanew)], columns=['NO_PURCHASE'] + slotsOffered + ['Discount', 'Eco', 'Gr'])
            predictprobdf.to_csv(Location + filename + 'predprobfeatures.csv')
            betadf.to_csv(Location + filename + 'betafeatures.csv')
            del summary, predictprobdf, designdf, featuresleveldf, assortmentdf, choicedf, design, featureslevel, assortment, choice
            break
    print("iteration: {}".format(i))
    print(betadf.shape)
    if i == 1001:
        return False
    return True
    #return betadf.iloc[0]

In [ ]:
def getHourList(zone):
    if zone == '500.0':
            hourlist={'06:30 - 08:00':'2',
           '08:00 - 10:00':'3',
           '10:00 - 12:00':'4',
           '12:00 - 14:00':'5',
           '14:00 - 16:00':'6',
           '16:00 - 18:00':'7',
           '18:00 - 20:00':'8',
           '20:00 - 22:00':'9',
           '22:00 - 23:30':'10'}
    else:
            hourlist={'05:00 - 06:00':'1',
           '06:00 - 08:00':'2',
           '08:00 - 10:00':'3',
           '10:00 - 12:00':'4',
           '12:00 - 14:00':'5',
           '14:00 - 16:00':'6',
           '16:00 - 18:00':'7',
           '18:00 - 20:00':'8',
           '20:00 - 22:00':'9',
           '22:00 - 23:30':'10'}
    return hourlist
daylist={'2':'1','3':'2','4':'3','5':'4','6':'5','0':'6','1':'7'} # no use

In [ ]:
if __name__ == "__main__":
    final = []
    zone = "500.0"
    hourlist=getHourList(zone)
    slotsOfferedRank=[]
    for i in range (1,8):
        for j in list(hourlist.values()):
                slotsOfferedRank = np.append(slotsOfferedRank,str(i)+'_'+j)
    zone_path, zone_file_prefix = get_zone_output_path(zone, root)
    summary = pd.read_csv('/content/drive/MyDrive/mm_probit/results/new_gr/simulation/500.0/gr_unranked_stacked_1500_arrival_0_cut_0_cap_0.csv')
    #summary = summary.drop(['NO_PURCHASE_Eco', 'NO_PURCHASE_Discount', 'NO_PURCHASEGr'], axis=1)
    slots_active = ['0_06:30 - 08:00','0_08:00 - 10:00','0_10:00 - 12:00','0_12:00 - 14:00', '0_14:00 - 16:00', '0_16:00 - 18:00', '0_18:00 - 20:00', '0_20:00 - 22:00', '0_22:00 - 23:30', '1_06:30 - 08:00', '1_08:00 - 10:00', '1_10:00 - 12:00', '1_12:00 - 14:00', '1_14:00 - 16:00', '1_16:00 - 18:00', '1_18:00 - 20:00', '1_20:00 - 22:00', '1_22:00 - 23:30', '2_06:30 - 08:00', '2_08:00 - 10:00', '2_10:00 - 12:00', '2_12:00 - 14:00', '2_14:00 - 16:00', '2_16:00 - 18:00', '2_18:00 - 20:00', '2_20:00 - 22:00', '2_22:00 - 23:30', '3_06:30 - 08:00', '3_08:00 - 10:00', '3_10:00 - 12:00', '3_12:00 - 14:00', '3_14:00 - 16:00', '3_16:00 - 18:00', '3_18:00 - 20:00', '3_20:00 - 22:00', '3_22:00 - 23:30', '4_06:30 - 08:00', '4_08:00 - 10:00', '4_10:00 - 12:00', '4_12:00 - 14:00', '4_14:00 - 16:00', '4_16:00 - 18:00', '4_18:00 - 20:00', '4_20:00 - 22:00', '4_22:00 - 23:30', '5_06:30 - 08:00', '5_08:00 - 10:00', '5_10:00 - 12:00', '5_12:00 - 14:00', '5_14:00 - 16:00', '5_16:00 - 18:00', '5_18:00 - 20:00', '5_20:00 - 22:00', '5_22:00 - 23:30', '6_06:30 - 08:00', '6_08:00 - 10:00', '6_10:00 - 12:00', '6_12:00 - 14:00', '6_14:00 - 16:00', '6_16:00 - 18:00', '6_18:00 - 20:00', '6_20:00 - 22:00', '6_22:00 - 23:30']
    slots_active_filtered = []
    for slot in slotsOfferedRank:
        sum_of_all_rows = summary[slot].sum()
        if sum_of_all_rows > 0:
            slots_active_filtered.append(slot)
            try:
                summary[slot+'_Discount']
            except:
                summary[slot+'_Discount'] = 0
    print(len(slots_active_filtered))
    print(slots_active_filtered)
    MMfeaturesBoot(zone_path, zone_file_prefix, summary, slots_active_filtered)

63
['1_2', '1_3', '1_4', '1_5', '1_6', '1_7', '1_8', '1_9', '1_10', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8', '2_9', '2_10', '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8', '3_9', '3_10', '4_2', '4_3', '4_4', '4_5', '4_6', '4_7', '4_8', '4_9', '4_10', '5_2', '5_3', '5_4', '5_5', '5_6', '5_7', '5_8', '5_9', '5_10', '6_2', '6_3', '6_4', '6_5', '6_6', '6_7', '6_8', '6_9', '6_10', '7_2', '7_3', '7_4', '7_5', '7_6', '7_7', '7_8', '7_9', '7_10']
normalizing
0.5210358314818356
-4.408548975262699
Iteration= 1 loglikelihood = nan beta_disc nan beta_eco nan beta_gr nan
nan
Iteration= 2 loglikelihood = nan beta_disc nan beta_eco nan beta_gr nan
nan
Iteration= 3 loglikelihood = nan beta_disc nan beta_eco nan beta_gr nan
nan
Iteration= 4 loglikelihood = nan beta_disc nan beta_eco nan beta_gr nan
nan
Iteration= 5 loglikelihood = nan beta_disc nan beta_eco nan beta_gr nan
nan
Iteration= 6 loglikelihood = nan beta_disc nan beta_eco nan beta_gr nan
nan
Iteration= 7 loglikelihood = nan beta_disc

KeyboardInterrupt: 

In [ ]:
#gr_df = pd.read_csv('gr_disc_35.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# slots_active = []
# for col in gr_df.columns:
#     if col.endswith('Gr'):
#         slots_active.append(col[:-2])
# print(slots_active)

In [ ]:
#put grs and disc to zero
def modify_summary(df, non_zero_slot):
    for col in df.columns:
        if col != non_zero_slot:
            zero_gr_col = col+'Gr'
            if zero_gr_col in df.columns:
                df[zero_gr_col].values[:] = 0
    return df

def modify_summary_incremental(df, non_zero_slots):
    for col in df.columns:
        if col.endswith('Gr') and col not in non_zero_slots:
            df[col].values[:] = 0
    return df

def check_summary(df):
    print("checking summary")
    for col in df.columns:
        if col.endswith('Gr'):
            if df[col].sum() != 0:
               print("non_zero_gr_col: " + col)

In [ ]:
## TODO: 1 probit on all columns at once, instead of separately with minimum Discount >= 35
## 2) Plain MM every arrival day before cut1 and before cut2 without Gr and with Gr
## 3) for each arrival day and for each cap_mode_choice for both zone 500,700
## 4) Bootstrap for standard errors (refer Fresh Direct notebook for details)



if __name__ == "__main__":
    zone_path, zone_file_prefix = get_zone_output_path(zone, root)
    #summary = pd.read_csv('/Users/anupamtripathi/PycharmProjects/RA_/results/gr/500.0/gr_unranked_stacked_arrival_0_cut_0_cap_2.csv')
    summary = pd.read_csv('gr_unranked_stacked_1500_arrival_0_cut_0_cap_0 (1).csv')
    #summary = pd.read_csv('/Users/sathvik/Desktop/RA/RA_/results/gr/active_all_slots/without_no_purchase/700.0/gr_unranked_stacked_arrival_0_cut_1_cap_0.csv')
    #summary = summary.drop(['NO_PURCHASE_Eco', 'NO_PURCHASE_Discount'], axis=1)
    summary = summary.fillna(0)
    # for col in summary.columns:
    #     print(col)
    # double checking slots_active is correctly calculated
    slots_active_filtered = []
    slots_active = ['0_06:30 - 08:00','0_08:00 - 10:00','0_10:00 - 12:00','0_12:00 - 14:00', '0_14:00 - 16:00', '0_16:00 - 18:00', '0_18:00 - 20:00', '0_20:00 - 22:00', '0_22:00 - 23:30', '1_06:30 - 08:00', '1_08:00 - 10:00', '1_10:00 - 12:00', '1_12:00 - 14:00', '1_14:00 - 16:00', '1_16:00 - 18:00', '1_18:00 - 20:00', '1_20:00 - 22:00', '1_22:00 - 23:30', '2_06:30 - 08:00', '2_08:00 - 10:00', '2_10:00 - 12:00', '2_12:00 - 14:00', '2_14:00 - 16:00', '2_16:00 - 18:00', '2_18:00 - 20:00', '2_20:00 - 22:00', '2_22:00 - 23:30', '3_06:30 - 08:00', '3_08:00 - 10:00', '3_10:00 - 12:00', '3_12:00 - 14:00', '3_14:00 - 16:00', '3_16:00 - 18:00', '3_18:00 - 20:00', '3_20:00 - 22:00', '3_22:00 - 23:30', '4_06:30 - 08:00', '4_08:00 - 10:00', '4_10:00 - 12:00', '4_12:00 - 14:00', '4_14:00 - 16:00', '4_16:00 - 18:00', '4_18:00 - 20:00', '4_20:00 - 22:00', '4_22:00 - 23:30', '5_06:30 - 08:00', '5_08:00 - 10:00', '5_10:00 - 12:00', '5_12:00 - 14:00', '5_14:00 - 16:00', '5_16:00 - 18:00', '5_18:00 - 20:00', '5_20:00 - 22:00', '5_22:00 - 23:30', '6_06:30 - 08:00', '6_08:00 - 10:00', '6_10:00 - 12:00', '6_12:00 - 14:00', '6_14:00 - 16:00', '6_16:00 - 18:00', '6_18:00 - 20:00', '6_20:00 - 22:00', '6_22:00 - 23:30']
    for slot in slots_active:
        #print(slot +  " sum of all rows: " + str( summary[slot].sum() ) )
        sum_of_all_rows = summary[slot].sum()
        if sum_of_all_rows > 0:
            slots_active_filtered.append(slot)
    slots_active_filtered_again=[]
    for slot in slots_active_filtered:
        if slot+'Gr' in summary.columns:
            slots_active_filtered_again.append(slot)
    MMfeaturesBoot(zone_path, zone_file_prefix, summary, slots_active_filtered_again)

    slot_to_convergence_map = {}
    for slot in slots_active_filtered_again:
        new_summary = summary.copy()
        new_summary = modify_summary(new_summary, slot)
        check_summary(new_summary)
        slot_to_convergence_map[slot] = MMfeaturesBoot(zone_path, zone_file_prefix, new_summary, slots_active_filtered_again)


KeyError: '0_06:30 - 08:00'

In [ ]:
slot_to_convergence_map

{'0_06:30 - 08:00': False,
 '0_08:00 - 10:00': False,
 '0_10:00 - 12:00': False,
 '0_12:00 - 14:00': False,
 '0_14:00 - 16:00': False,
 '0_16:00 - 18:00': False,
 '0_18:00 - 20:00': False,
 '0_20:00 - 22:00': False,
 '0_22:00 - 23:30': True,
 '1_06:30 - 08:00': True,
 '1_08:00 - 10:00': True,
 '1_10:00 - 12:00': False,
 '1_12:00 - 14:00': True,
 '1_14:00 - 16:00': True,
 '1_16:00 - 18:00': False,
 '1_18:00 - 20:00': True,
 '1_20:00 - 22:00': True,
 '1_22:00 - 23:30': False,
 '2_06:30 - 08:00': True,
 '2_08:00 - 10:00': True,
 '2_10:00 - 12:00': True,
 '2_12:00 - 14:00': False,
 '2_14:00 - 16:00': True,
 '2_16:00 - 18:00': True,
 '2_18:00 - 20:00': True,
 '2_20:00 - 22:00': True,
 '2_22:00 - 23:30': True,
 '3_06:30 - 08:00': False,
 '3_08:00 - 10:00': False,
 '3_10:00 - 12:00': False,
 '3_12:00 - 14:00': True,
 '3_14:00 - 16:00': False,
 '3_16:00 - 18:00': False,
 '3_18:00 - 20:00': False,
 '3_20:00 - 22:00': True,
 '3_22:00 - 23:30': True,
 '4_06:30 - 08:00': False,
 '4_08:00 - 10:00':

In [ ]:
slot_to_convergence_map

In [ ]:
coverging_slots = []
non_converging_slots = []
for k,v in slot_to_convergence_map.items():
    if v == True:
        coverging_slots.append(k)
    else:
        non_converging_slots.append(k)

In [ ]:
coverging_slots = [slot+'Gr' for slot in coverging_slots]

In [ ]:
zone_path, zone_file_prefix = get_zone_output_path(zone, root)
#summary = pd.read_csv('/Users/anupamtripathi/PycharmProjects/RA_/results/gr/500.0/gr_unranked_stacked_arrival_0_cut_0_cap_2.csv')
summary = pd.read_csv('results/gr/700.0/gr_unranked_stacked_arrival_0_cut_0_cap_0.csv')
#summary = pd.read_csv('/Users/sathvik/Desktop/RA/RA_/results/gr/active_all_slots/without_no_purchase/700.0/gr_unranked_stacked_arrival_0_cut_1_cap_0.csv')
summary = summary.drop(['NO_PURCHASE_Eco', 'NO_PURCHASE_Discount'], axis=1)
summary = summary.fillna(0)

 # double checking slots_active is correctly calculated
slots_active_filtered = []
for slot in slots_active:
    #print(slot +  " sum of all rows: " + str( summary[slot].sum() ) )
    sum_of_all_rows = summary[slot].sum()
    if sum_of_all_rows > 0:
        slots_active_filtered.append(slot)

non_zero_gr_cols = []
zero_gr_cols = coverging_slots
non_zero_gr_cols.append(zero_gr_cols.pop(0))

slots_increment_conveging_map  = {}
while len(non_zero_gr_cols) != 0 and len(zero_gr_cols) != 0:
    new_summary = summary.copy()
    new_summary = modify_summary_incremental(new_summary, non_zero_gr_cols)
    check_summary(new_summary)
    #print(non_zero_gr_cols)
    non_zero_gr_cols.append(zero_gr_cols.pop(0))
    non_zero_gr_cols_str = ('_').join(non_zero_gr_cols)
    slots_increment_conveging_map[non_zero_gr_cols_str] = MMfeaturesBoot(zone_path, zone_file_prefix, new_summary, slots_active_filtered)


In [ ]:
# 5/3 - Tues
# TODO
# 1) Split the slots as converging and non-converging gr slots
# 2) Keep incrementally adding only converging slots and see overall convergence
# 3) try analyze properties of non-converging time slots
# 4) check linear dependence between gr cols

In [ ]:
slots_increment_conveging_map

In [ ]:
non_converging_gr_slots = [slot+'Gr' for slot in non_converging_slots]
summary_non_converging_gr_slots = summary[non_converging_gr_slots]

In [ ]:
non_gr_summary = summary_non_converging_gr_slots.describe()
non_gr_summary.to_csv('non_gr_summary.csv')

In [ ]:
corrrelation    = summary_non_converging_gr_slots.corr(method="pearson");
print("Pearson correlation coefficient:");
print(corrrelation);
corrrelation.to_csv("pearson.csv")
plt.figure(figsize = (15,8))
sns.heatmap(corrrelation,cmap="YlGnBu")
plt.savefig('pearson_heatmap.png')

In [ ]:
corrrelation    = summary_non_converging_gr_slots.corr(method="spearman");
print("Spearman rank correlation:");
corrrelation.to_csv("spearman.csv")
sns.heatmap(corrrelation,cmap="YlGnBu")
print(corrrelation);